In [ ]:
from scr import *
pd.set_option('display.max_rows', None)

### scraping twitter

In [ ]:
# palabras clave

triste = ('solo','pistola','muerto','muerte','ansioso','avergonzado','sin animo','llorar','depreciado','desesperado','desmotivado','deprimido','desolado','desesperado','devastado','decepcionado', "desanimado", "dolor","angustiado", "fatigado", "temeroso", "asustado", "frustrado", "sombrio", "afligido", "melancolico", "sin esperanza", "humillado", "ignorado", "intimidado", "insomnio", "aislado", "matar","morir", "solitario", "salud mental", "maltratado", "triste", "nervioso", "ofendido", "oprimido", "abrumado", "panico", "castigado", "triste", "miedo", 'tristeza','arrepentido','estresado','suicidio','suicidar','sufrimiento','lloro','aterrorizado','cansado','tragico','incomodo','infeliz','molesto','inutil','victima''vulnerable','debil','lloroso','preocupado')
feliz = ('vivo','asombroso','impresionante','bendecido','dichoso','impresionado','animado','despreocupado','encantador','alegre', "animado", "contento", "encantado", "euforico", "exultante", "fantastico", "fascinado", "halagado", "realizado", "alegre", "feliz", "increible", "amor", "afortunado", "maravilloso", "alegremente", "alucinado", "motivado", "desbordante", "exultante", "contento", "relajado", "aliviado", "satisfecho", "sensacional", "sonriente", "sorprendido", "entusiasmado", "triunfante", "confiado", "animado", "victorioso", "maravilloso")

In [ ]:
lista = triste
max_tweets = 10000
since = '2022-09-20'
until = '2022-09-21'
start_time = datetime.now()

sept_20 = scraper(lista,max_tweets,since,until)
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

sept_20.to_csv("sept_20.csv")

Duration: 0:06:42.370216


In [ ]:
lista_triste = (
sept_07,sept_08,sept_09,sept_10,sept_11,sept_12,sept_13,sept_14,
sept_15,sept_16,sept_17,sept_18,sept_19,sept_20
)

triste_sept = pd.concat(lista_triste)
triste_sept.to_csv("triste_sept.csv")

### cleaning


In [ ]:
triste_sept.drop(['url',
                  'renderedContent',
                  'replyCount',
                  'retweetCount',
                  'likeCount',
                  'quoteCount',
                  'conversationId',
                  'lang',
                  'source',
                  'sourceUrl',
                  'sourceLabel',
                  'links',
                  'media',
                  'retweetedTweet',
                  'quotedTweet',
                  'inReplyToTweetId',
                  'inReplyToUser',
                  'mentionedUsers',
                  'coordinates',
                  'place',
                  'hashtags',
                  'cashtags',
                  'card'
                 ],axis=1,inplace=True)

In [ ]:
triste_sept = triste_sept.rename({'rawContent':'Text'},axis=1)

In [ ]:
triste_sept.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 140000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype              
---  ------  --------------   -----              
 0   date    140000 non-null  datetime64[ns, UTC]
 1   Text    140000 non-null  object             
 2   id      140000 non-null  int64              
 3   user    140000 non-null  object             
dtypes: datetime64[ns, UTC](1), int64(1), object(2)
memory usage: 5.3+ MB


In [ ]:
def clean_text(text):
    text_cleaning_re = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"
    text = re.sub(text_cleaning_re, ' ', str(text).lower()).strip()

    tokens = text.split()
    stop_words = set(stopwords.words('spanish'))
    tokens = [w for w in tokens if w not in stop_words]
    return " ".join(tokens)

In [ ]:
# paso la columna text a lista
tweets_lista_s = triste_sept['Text'].tolist()
# aplico la funcion remove_username_links
removed_s = [remove_usernames_links(tweet) for tweet in tweets_lista_s]
# aplico la funcion clean_tweet
cleaned_s = [clean_tweet(tweet) for tweet in removed_s]
# aplico la funcion no emoji
emojis_s = [no_emoji(tweet) for tweet in cleaned_s]
# pongo el contenido de la lista en una nueva columna
triste_sept['Tweets'] = emojis_s

### sentiment analysis

In [ ]:
from sentiment_analysis_spanish import sentiment_analysis

In [ ]:
def positive_negative(df):
    for item in df:
        if item >= 0:
            Sentiment.append("Positive")
        elif item < 0:
            Sentiment.append("Negative")

In [ ]:
def sentiment_scores(frase):
    '''
    Funcion que calcula el score de sentimiento
    en español.
    '''
    sentiment = sentiment_analysis.SentimentAnalysisSpanish()
    return sentiment.sentiment(frase)

In [ ]:
triste_clean.drop(['Text'],axis=1,inplace=True)

start_time = datetime.now()
triste_clean['Polarity'] = [sentiment_scores(i) for i in triste_clean['Tweets']]
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

In [ ]:
Sentiment = []
positive_count_sad = 0
negative_count_sad = 0

for item in sad_posneg['Polarity']:
    if item >= 0:
        positive_count_sad +=1
        Sentiment.append("Positivo")
    elif item < 0:
        negative_count_sad +=1
        Sentiment.append("Negativo")

se = pd.Series(Sentiment)
triste_polarity['Sentiment'] = se.values

In [ ]:
label_enc = LabelEncoder()
senti_transformada = label_enc.fit_transform(triste_polarity['Sentiment'])
triste_polarity['Encoding'] = senti_transformada